In [2]:
import pandas as pd

pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
pth_offers = "/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n.csv"

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])

df_offers

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5
0,Портативная акустика Hoco HC2 Xpress темно-зел...,4771613,портативная акустика,3904,4771613,29.91,4851243,27.94,4777265,22.92,4851242,22.30,5491278,21.59
1,Акустическая система HYUNDAI Колонка порт. H-P...,4570006,портативная акустика,3904,4569993,24.89,4569999,24.86,4569990,24.72,3709109,24.70,4569996,24.69
2,"Смартфон Xiaomi POCO M5 4/128 GB Global, серый",5506115,мобильные телефоны,2801,5506115,27.33,5667654,26.23,5667653,21.38,5254579,21.25,5506116,21.23
3,"Портативная колонка Ginzzu GM-906B, BT-Колонка...",4585102,портативная акустика,3904,4585102,28.42,1017511,24.23,939517,24.12,1431431,23.80,3033277,23.39
4,Часы Amazfit BIP S Black,4565835,смарт-часы и браслеты,2815,4565835,28.23,4565836,23.02,4565837,19.52,3390599,19.11,5418954,18.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76694,"Акустика портативная Borofone BR6 Miraculous, ...",4569883,портативная акустика,3904,4569883,28.00,4851048,19.95,4569882,19.77,4851045,19.77,6957042,19.39
76695,Часы с GPS трекером Digma Kid K7m Yellow/Blue,4179687,смарт-часы и браслеты,2815,4179687,27.73,4753900,19.11,4777070,18.89,3855791,18.89,4179703,17.88
76696,Мобильный телефон INFINIX HOT 12 Pro 8+128GB Blue,5492470,мобильные телефоны,2801,5492470,29.50,5821454,26.39,5145759,22.38,5821453,21.95,5137688,20.39
76697,"Умная колонка MAIL Капсула, белый [mrc01(white)]",4855069,портативная акустика,3904,4855069,24.73,6226397,23.33,4854468,22.78,4048269,15.95,3938298,15.45


In [3]:
"""
Случаи в данных с ошибками:

0) исключить из ручной проверки совпадающие top_1-top_5 с разметкой model_id. Если top_1 != model_id, ColBERT ошибся.
Пренебрегаем случаями когда разметка неправильная и среди top_1-top_5 находится этот неправильный model_id. Допустим что их количество мало.
5144478: Samsung Galaxy Tab S8
Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64

1) ColBERT находит среди top_1-top_5 правильный матч, а разметка model_id при этом не правильная.
Тогда пользователь руками вводит число от 1 до 5, записывается новый model_id 
Эта ошибка не должна учитываться в метрике accuracy, ее необходимо исключить из тестового датасета.
5144478: Samsung Galaxy Tab S9
Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64

2) среди top_1-top_5 нет правильного матча, разметка model_id правильная
5144478: Samsung Galaxy Tab S8
Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S9 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64

3) среди top_1-top_5 нет правильного матча, разметка model_id не правильная
Эта ошибка не должна учитываться в метрике accuracy, ее необходимо исключить из тестового датасета.
5144478: Samsung Galaxy Tab S9
Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S10 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64
"""

"""
Действия:
I) Заводим колонки model_id_correct, model_id_manual
II) Делаем прогон да/нет-кой на строках с top_1-top_5 != model_id, записываем в model_id_correct 0/1/2. 0-неправильно, 1-правильно, 2-хз
III) Для неправильной разметки переразмечаем см случай 1), если среди top_1-top_5 нет правильного матча см случай 3) ставим model_id_manual = 0
IV) Для правильной разметки см случай 2) не исключаем примеры, они должны участвовать в ошибке

"""

"""
- остальные 2334шт выкинуть из val, посчитать метрики без этих примеров
- разметить model_id=0 ColBERT'ом, кластеризовать, посмотреть, прокликать да/нет-кой
- завести задачу для Гриши
"""

"\n- из 1000 шт да/нет-ки выделить случаи не правильной разметки и посчитать долю правильно предсказанных ColBERT'ом\n- остальные 2334шт выкинуть из val, посчитать метрики без этих примеров\n- разметить model_id=0 ColBERT'ом, кластеризовать, посмотреть, прокликать да/нет-кой\n- завести задачу для Гриши\n"

In [4]:
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
df = df_offers.copy()
for i in range(len(clms)):
    df = exclude_from_df(df, clms[i])
# df.to_csv('/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n_exclude_top5.csv', sep=';', index=False)
df

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5
0,Акустическая система HYUNDAI Колонка порт. H-P...,4570006,портативная акустика,3904,4569993,24.89,4569999,24.86,4569990,24.72,3709109,24.70,4569996,24.69
1,Смарт-часы CANYON CNS-SW82BB,4565865,смарт-часы и браслеты,2815,4178947,27.38,5151627,26.16,4178942,25.06,4178944,23.80,5151628,23.55
2,Гарнитура Мультимедийные стереонаушники с микр...,1001102,"наушники, гарнитуры, наушники c микрофоном",2102,3621627,16.88,4636867,16.78,2050212,16.75,4735667,16.73,1426187,16.70
3,SVEN SV-015008,1426186,"наушники, гарнитуры, наушники c микрофоном",2102,4455941,24.12,1034729,22.95,4770097,22.52,1040395,22.50,2083909,22.47
4,Гарнитура CANYON проводные наушники с микрофон...,4676812,"наушники, гарнитуры, наушники c микрофоном",2102,749928,21.75,2986233,21.64,3537855,21.52,746343,21.48,933005,21.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,Внутриканальные (вакуумные) беспроводные Focal...,1426118,"наушники, гарнитуры, наушники c микрофоном",2102,4455962,17.16,746174,16.80,4677040,16.77,4300242,16.64,4300243,16.61
3330,Bluetooth стерео-гарнитура Harper Наушники с м...,4049393,"наушники, гарнитуры, наушники c микрофоном",2102,4455800,23.61,4533126,23.58,4533125,23.53,4994477,23.52,746520,23.52
3331,Гарнитура GEMBIRD проводные наушники с микрофо...,742577,"наушники, гарнитуры, наушники c микрофоном",2102,742576,22.56,4049936,22.56,742571,22.48,742581,22.45,742579,22.41
3332,2в1 Портативная акустика + внешний аккумулятор...,4851344,портативная акустика,3904,300218,15.62,332059,15.11,290713,14.83,705129,14.81,297140,14.73


In [5]:
df[df.name == "Планшетный компьютер Apple iPad Pro 12.9-inch Wi-Fi + Cellular 512GB - Silver [MXF82RU/A] (2020)"]

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5
751,Планшетный компьютер Apple iPad Pro 12.9-inch ...,4355166,планшетные компьютеры и мини-планшеты,510401,4070863,25.69,4355160,25.69,4070867,25.66,1465229,25.59,4800947,25.58


In [6]:
dict(df.iloc[390])

{'name': 'Планшет Apple 11-inch iPad Pro Wi-Fi + Cellular 256GB, Silver (MXE52RU/A)',
 'model_id': 911280,
 'category_name': 'планшетные компьютеры и мини-планшеты',
 'category_id': 510401,
 'model_id_pred_1': 4069038,
 'similarity_1': 28.39,
 'model_id_pred_2': 4800942,
 'similarity_2': 28.08,
 'model_id_pred_3': 4355157,
 'similarity_3': 27.98,
 'model_id_pred_4': 4800941,
 'similarity_4': 24.56,
 'model_id_pred_5': 5811841,
 'similarity_5': 24.28}

In [7]:
for index, row in df[751:752].iterrows():
    model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
    model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
    print(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")  
    model_pred = list(df_models[df_models.model_id == row['model_id_pred_2']]['full_name'])[0]
    print(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")  
    model_pred = list(df_models[df_models.model_id == row['model_id_pred_3']]['full_name'])[0]
    print(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")  
    model_pred = list(df_models[df_models.model_id == row['model_id_pred_4']]['full_name'])[0]
    print(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")  


Apple iPad Pro 12.9 (2020) 512GB WiFi + Cellular:
Планшетный компьютер Apple iPad Pro 12.9-inch Wi-Fi + Cellular 512GB - Silver [MXF82RU/A] (2020) --> Apple iPad Pro 12.9 (2018) 256Gb Wi-Fi


Apple iPad Pro 12.9 (2020) 512GB WiFi + Cellular:
Планшетный компьютер Apple iPad Pro 12.9-inch Wi-Fi + Cellular 512GB - Silver [MXF82RU/A] (2020) --> Apple iPad Pro 12.9 (2020) 128Gb Wi-Fi


Apple iPad Pro 12.9 (2020) 512GB WiFi + Cellular:
Планшетный компьютер Apple iPad Pro 12.9-inch Wi-Fi + Cellular 512GB - Silver [MXF82RU/A] (2020) --> Apple iPad Pro 12.9 (2018) 64Gb Wi-Fi


Apple iPad Pro 12.9 (2020) 512GB WiFi + Cellular:
Планшетный компьютер Apple iPad Pro 12.9-inch Wi-Fi + Cellular 512GB - Silver [MXF82RU/A] (2020) --> Apple iPad Pro 12.9 (2017) 64Gb Wi-Fi



In [8]:
dict(df.iloc[0])

{'name': 'Акустическая система HYUNDAI Колонка порт. H-PAC560 черный 10W 1.0 BT / 3.5Jack / USB 10м 3000mAh',
 'model_id': 4570006,
 'category_name': 'портативная акустика',
 'category_id': 3904,
 'model_id_pred_1': 4569993,
 'similarity_1': 24.89,
 'model_id_pred_2': 4569999,
 'similarity_2': 24.86,
 'model_id_pred_3': 4569990,
 'similarity_3': 24.72,
 'model_id_pred_4': 3709109,
 'similarity_4': 24.7,
 'model_id_pred_5': 4569996,
 'similarity_5': 24.69}

In [9]:
def choose_0_5(df, df_models):
    for index, row in df.iterrows():
        model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
        top_1 = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
        top_2 = list(df_models[df_models.model_id == row['model_id_pred_2']]['full_name'])[0]
        top_3 = list(df_models[df_models.model_id == row['model_id_pred_3']]['full_name'])[0]
        top_4 = list(df_models[df_models.model_id == row['model_id_pred_4']]['full_name'])[0]
        top_5 = list(df_models[df_models.model_id == row['model_id_pred_5']]['full_name'])[0]
        print(f"\n{model_gt}:\n{row['name']}")  
        print(f"1) {top_1}\n2) {top_2}\n3) {top_3}\n4) {top_4}\n5) {top_5}\n")

        # Запрос ввода от пользователя
        manual_input = input("Введите число от 0 до 5: ")
        
        # Проверка введенного значения и вставка соответствующего значения в колонку "manual"
        if manual_input.isdigit():
            manual_input = int(manual_input)
            if manual_input == 0:
                df.at[index, 'model_id_manual'] = 0
            elif manual_input >= 1 and manual_input <= 5:
                df.at[index, 'model_id_manual'] = row[f'model_id_pred_{manual_input}']
            else:
                print("Некорректный ввод. Введите число от 0 до 5.")
        else:
            print("Некорректный ввод. Введите число от 0 до 5.")
    return df

# df1 = choose_0_5(df[:10], df_models)
# df1

In [10]:
df

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5
0,Акустическая система HYUNDAI Колонка порт. H-P...,4570006,портативная акустика,3904,4569993,24.89,4569999,24.86,4569990,24.72,3709109,24.70,4569996,24.69
1,Смарт-часы CANYON CNS-SW82BB,4565865,смарт-часы и браслеты,2815,4178947,27.38,5151627,26.16,4178942,25.06,4178944,23.80,5151628,23.55
2,Гарнитура Мультимедийные стереонаушники с микр...,1001102,"наушники, гарнитуры, наушники c микрофоном",2102,3621627,16.88,4636867,16.78,2050212,16.75,4735667,16.73,1426187,16.70
3,SVEN SV-015008,1426186,"наушники, гарнитуры, наушники c микрофоном",2102,4455941,24.12,1034729,22.95,4770097,22.52,1040395,22.50,2083909,22.47
4,Гарнитура CANYON проводные наушники с микрофон...,4676812,"наушники, гарнитуры, наушники c микрофоном",2102,749928,21.75,2986233,21.64,3537855,21.52,746343,21.48,933005,21.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,Внутриканальные (вакуумные) беспроводные Focal...,1426118,"наушники, гарнитуры, наушники c микрофоном",2102,4455962,17.16,746174,16.80,4677040,16.77,4300242,16.64,4300243,16.61
3330,Bluetooth стерео-гарнитура Harper Наушники с м...,4049393,"наушники, гарнитуры, наушники c микрофоном",2102,4455800,23.61,4533126,23.58,4533125,23.53,4994477,23.52,746520,23.52
3331,Гарнитура GEMBIRD проводные наушники с микрофо...,742577,"наушники, гарнитуры, наушники c микрофоном",2102,742576,22.56,4049936,22.56,742571,22.48,742581,22.45,742579,22.41
3332,2в1 Портативная акустика + внешний аккумулятор...,4851344,портативная акустика,3904,300218,15.62,332059,15.11,290713,14.83,705129,14.81,297140,14.73


In [14]:
pth_1000 = "/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n_exclude_top5_yes_no_1000.csv"

df_1000 = pd.read_csv(pth_1000, sep=";")
df_1000 = df_1000[:1000]
df_1000

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_correct
0,Акустическая система HYUNDAI Колонка порт. H-P...,4570006,портативная акустика,3904,4569993,24.89,4569999,24.86,4569990,24.72,3709109,24.70,4569996,24.69,1.0
1,Смарт-часы CANYON CNS-SW82BB,4565865,смарт-часы и браслеты,2815,4178947,27.38,5151627,26.16,4178942,25.06,4178944,23.80,5151628,23.55,2.0
2,Гарнитура Мультимедийные стереонаушники с микр...,1001102,"наушники, гарнитуры, наушники c микрофоном",2102,3621627,16.88,4636867,16.78,2050212,16.75,4735667,16.73,1426187,16.70,1.0
3,SVEN SV-015008,1426186,"наушники, гарнитуры, наушники c микрофоном",2102,4455941,24.12,1034729,22.95,4770097,22.52,1040395,22.50,2083909,22.47,2.0
4,Гарнитура CANYON проводные наушники с микрофон...,4676812,"наушники, гарнитуры, наушники c микрофоном",2102,749928,21.75,2986233,21.64,3537855,21.52,746343,21.48,933005,21.39,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Универсальная мобильная гарнитура JBL C50HI RE...,4455819,"наушники, гарнитуры, наушники c микрофоном",2102,746901,15.19,4300186,15.00,616609,14.67,760225,14.57,4300180,14.42,1.0
996,Планшетный компьютер Digma Планшет Citi 7591 3...,4273003,планшетные компьютеры и мини-планшеты,510401,3843911,25.70,4273002,25.31,1950145,25.25,4404425,25.22,2341722,25.12,1.0
997,"внутриканальные Наушники Xiaomi BHR5264GL, Mi ...",4455970,"наушники, гарнитуры, наушники c микрофоном",2102,5137672,20.44,4677044,20.28,1041951,20.22,5137673,19.95,4049806,19.94,2.0
998,Портативная акустическая система Hyundai Колон...,4570004,портативная акустика,3904,4569993,24.36,4569996,24.23,4569999,24.23,3709109,24.17,4585106,24.17,1.0


In [32]:
df_1000_errors_2 = df_1000[df_1000.model_id_correct == 2.0]
df_1000_errors_2.to_csv('/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n_exclude_top5_yes_no_1000_errors_2.csv', sep=';', index=False)

df_1000_errors_2

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_correct
1,Смарт-часы CANYON CNS-SW82BB,4565865,смарт-часы и браслеты,2815,4178947,27.38,5151627,26.16,4178942,25.06,4178944,23.80,5151628,23.55,2.0
3,SVEN SV-015008,1426186,"наушники, гарнитуры, наушники c микрофоном",2102,4455941,24.12,1034729,22.95,4770097,22.52,1040395,22.50,2083909,22.47,2.0
6,Наушники HP T1A67AA,4533500,"наушники, гарнитуры, наушники c микрофоном",2102,3367309,23.94,4049448,22.72,746528,22.41,4883259,21.34,742770,20.95,2.0
8,Наушники Skullcandy S2IVW-N740,4731816,"наушники, гарнитуры, наушники c микрофоном",2102,856843,26.08,856844,26.03,4090100,26.00,4090102,25.75,4090099,25.70,2.0
19,Портативный GPS-навигатор Navitel,930885,GPS-навигаторы,280801,145480,26.36,145477,26.23,145828,26.20,145829,26.14,930883,26.12,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,Игровая гарнитура проводная Crown CMGH-3103 че...,4455769,"наушники, гарнитуры, наушники c микрофоном",2102,4455770,28.56,4533066,24.64,4533065,23.28,4533431,23.20,4533062,20.78,2.0
921,"Планшет DIGMA 1402D 10.1"" 4/64GB Silver Wi-Fi+...",4557928,планшетные компьютеры и мини-планшеты,510401,6177706,20.23,831852,20.20,1464879,19.67,646909,19.50,944123,19.23,2.0
925,Смартфон Acer V360 Dual SIM Black Acer Смартфо...,325207,мобильные телефоны,2801,190678,23.36,818797,23.30,188050,23.28,137982,23.22,938217,23.19,2.0
945,Радиоприёмник Sangean Revery R4,4569805,"радиоприемники, радиобудильники, радиочасы",3903,594083,23.64,4569804,21.84,4569801,20.88,512753,20.58,871021,19.81,2.0


In [49]:
id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

print(dict(df_1000.model_id_correct.value_counts()))

# for category in id_category.values():
#     print(category)
#     df_1000_cat = df_1000[df_1000.category_name == category]
#     print(len(df_1000_cat))
#     print(dict(df_1000_cat.model_id_correct.value_counts()))

{1.0: 775, 0.0: 150, 2.0: 75}


In [47]:
pth_1000_errors_2 = '/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n_exclude_top5_yes_no_1000_errors_2.csv'
df_1000_errors_2 = pd.read_csv(pth_1000_errors_2, sep=";")
# df_1000_errors_2 = df_1000_errors_2[df_1000_errors_2.category_id == 2102]
df_1000_errors_2['model_id_manual'] = df_1000_errors_2['model_id_manual'].astype(int)
df_1000_errors_no_model = df_1000_errors_2[df_1000_errors_2.model_id_manual == df_1000_errors_2.model_id_pred_5].reset_index(drop=True)
df_1000_errors_correct = df_1000_errors_2[df_1000_errors_2.model_id_manual == df_1000_errors_2.model_id_pred_1].reset_index(drop=True)
df_1000_errors_not_correct = df_1000_errors_2[df_1000_errors_2.model_id_manual == 0].reset_index(drop=True)

df_1000_errors_correct

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_correct,model_id_manual
0,Умные часы Smarterra Смарт-часы FitMaster Aura...,4271585,смарт-часы и браслеты,2815,4271583,28.09,4271584,25.06,5151697,24.28,4179708,24.08,3933671,23.80,2.0,4271583
1,Наушники Marley EM-FE033 White,1001031,"наушники, гарнитуры, наушники c микрофоном",2102,2812317,27.09,2812316,26.02,856735,25.88,950278,23.92,2812333,21.98,2.0,2812317
2,Портативная акустика Philips BT 110 Красный,3709126,портативная акустика,3904,995747,24.42,995746,23.39,291994,22.53,2705892,20.80,4851951,20.09,2.0,995747
3,Акустика портативная RITMIX SP-150,686059,портативная акустика,3904,4298653,29.72,1153071,26.91,302232,22.97,290907,22.77,4851351,22.72,2.0,4298653
4,Часы Apple Watch SE 2 GPS 44mm MNTE3 модель A2...,5678011,смарт-часы и браслеты,2815,4511020,25.06,4577121,24.05,4511071,23.72,5678012,23.48,5678594,22.88,2.0,4511020
5,"Планшет Apple iPad Pro 12.9 (2021), 8/256 Гб, ...",4800929,планшетные компьютеры и мини-планшеты,510401,4800951,28.09,4800952,25.36,4070863,25.05,1465233,24.97,4800941,24.25,2.0,4800951
6,Смартфон LG K7 (2017) X230 Titan,885408,мобильные телефоны,2801,2111564,26.72,2038470,24.75,112233,21.75,114776,21.52,116203,21.36,2.0,2111564
7,"Яндекс Умная колонка Яндекс.Станция Лайт, цвет...",4854469,портативная акустика,3904,4854470,27.98,5151961,27.42,5151970,27.22,5151966,27.20,5151965,27.19,2.0,4854470
8,Компьютерная гарнитура Jabra STEALTH UC черный...,747057,"наушники, гарнитуры, наушники c микрофоном",2102,636346,22.28,742818,22.03,747204,21.45,4300127,21.03,747222,20.97,2.0,636346
9,Apple Watch SE GPS 40мм Aluminum Case with Spo...,4511020,смарт-часы и браслеты,2815,4511029,30.28,4511035,28.89,4580479,27.98,5678011,26.34,5769430,25.44,2.0,4511029


In [46]:
df_1000_errors_not_correct

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_correct,model_id_manual
0,Беспроводные Bluetooth наушники для мобильных ...,4731862,"наушники, гарнитуры, наушники c микрофоном",2102,4802464,15.53,4533628,15.12,4636872,15.06,4533632,14.98,743101,14.97,2.0,0
1,Беспроводная акустика Умная колонка Яндекс.Ста...,4854469,портативная акустика,3904,5151970,21.69,5151961,21.66,5151964,21.66,5151967,21.59,5151963,21.52,2.0,0
2,"Bluetooth стерео-гарнитура, Спортивная Bluetoo...",4533214,"наушники, гарнитуры, наушники c микрофоном",2102,4153443,14.40,5490537,14.33,4754612,14.30,1034671,14.27,4735624,14.24,2.0,0
3,Полностью беспроводные наушники Bowers & Wilki...,4850060,"наушники, гарнитуры, наушники c микрофоном",2102,742049,21.72,4455762,21.52,742050,21.52,6200717,21.50,5137469,21.42,2.0,0
4,"Планшет DIGMA 1402D 10.1"" 4/64GB Silver Wi-Fi+...",4557928,планшетные компьютеры и мини-планшеты,510401,6177706,20.23,831852,20.20,1464879,19.67,646909,19.50,944123,19.23,2.0,0


In [48]:
print(f"ColBERT прав: {len(df_1000_errors_correct)}\nColBERT ошибся: {len(df_1000_errors_not_correct)}\nотсутствующая модель: {len(df_1000_errors_no_model)}")

ColBERT прав: 31
ColBERT ошибся: 5
отсутствующая модель: 39


In [ ]:
{1.0: 775, 0.0: 150, 2.0: 75}

# Для данных в которых среди top_5 матчей ColBERT'а нет model_id, присвоенного офферу моделайзером 
# Моделайзер прав: 775
# ColBERT прав: 31
# Моделайзер и ColBERT ошиблись одновременно: 5
# проблема с оффером или моделью: 189